In [5]:
%pip install finter pykrx python-dotenv pymysql mysql-connector-python sqlalchemy

  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached nbconvert-7.16.4-py3-none-any.whl.metadata (8.5 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached bleach-6.1.0-py3-none-any.whl.metadata (30 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached jupyterlab_pygments-0.3.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached MarkupSafe-2.1.5-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
  Using cached mistune-3.0.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached nbclient-0.10.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached pandocfilters-1.5.1-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached tinycss2-1.3.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached fastjsonschema-2.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
INFO: pip

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEME = os.getenv("DB_SCHEME")


In [7]:
from finter.framework_model.calendar import (
    iter_trading_days, iter_holidays, iter_days, TradingDay, Code
)


2024-10-06 14:13:13,450 - finter_sdk - INFO - API Key is not set. Your now using finter Open Version.

You can set the API Key in one of the following ways:
- By setting an environment variable directly in your environment:
    import os
    os.environ['FINTER_API_KEY'] = 'YOUR_API_KEY'

- By adding the following line to a .env file located in the project root:
    FINTER_API_KEY='YOUR_API_KEY'


[datetime.datetime(2023, 12, 25, 0, 0), datetime.datetime(2023, 12, 29, 0, 0)]

In [15]:
from datetime import datetime

current_year = datetime.now().year
first_day_of_year = int(f"{current_year}0101")
last_day_of_year = int(f"{current_year}1231")

dates = list(iter_holidays(first_day_of_year, last_day_of_year, exchange="krx"))

dates_to_string = [date.strftime('%Y-%m-%d') for date in dates]

In [27]:
from sqlalchemy import Column, String, create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class MarketHoliday(Base):
    __tablename__ = 'MARKET_HOLIDAY'
    
    MARKET_HOLIDAY_DATE = Column(String(10), primary_key=True)

engine = create_engine(f'mysql+pymysql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_SCHEME}')
Session = sessionmaker(bind=engine)
session = Session()

for date_str in dates_to_string:
    holiday = MarketHoliday(MARKET_HOLIDAY_DATE=date_str)
    session.merge(holiday)

session.commit()

print("데이터가 성공적으로 저장되었습니다!")

데이터가 성공적으로 저장되었습니다!
